In [ ]:
!pip install xpander-sdk boto3 python-dotenv

In [2]:
from xpander_sdk import XpanderClient, LLMProvider
import boto3
import os
import dotenv
import json
from IPython.display import Markdown

dotenv.load_dotenv()

session = boto3.Session(profile_name='prod')
bedrock_client = session.client('bedrock-runtime', region_name='us-west-2')
xpander_client = XpanderClient(api_key=os.getenv('XPANDER_API_KEY'))
agent1 = xpander_client.agents.get(agent_id=os.getenv('XPANDER_AGENT_ID'))
print(agent1.get_tools(llm_provider=LLMProvider.AMAZON_BEDROCK))


[{'toolSpec': {'name': 'PgLinkedInGetCompanyDetailsAndInsights', 'description': "This operation retrieves detailed insights and data about a specified company from LinkedIn. Use this operation when you need to analyze a company's profile, gather insights for market research, or evaluate potential business opportunities. It is particularly useful for professionals in sales, marketing, and business development who require comprehensive company information to inform their strategies.", 'inputSchema': {'json': {'type': 'object', 'properties': {'queryParams': {'type': 'object', 'properties': {}, 'required': []}, 'pathParams': {'type': 'object', 'properties': {}, 'required': []}, 'bodyParams': {'type': 'object', 'properties': {'inputTask': {'type': 'string', 'description': 'input user sub task based on the input query'}}, 'required': []}}, 'required': ['query_params', 'path_params', 'body_params']}}}}]


In [3]:
model_id = "anthropic.claude-3-5-sonnet-20241022-v2:0"

prompt = "Find information about xpander.ai and their main technology, summarize what they do and interesting facts about them."

user_message = {
    "role": "user",
    "content": [{"text": prompt}] 
}

memory = []
memory.append(user_message)
number_of_calls = 1

tool_config = {
    "tools": agent1.retrieve_all_graph_tools(llm_provider=LLMProvider.AMAZON_BEDROCK)
}

while True:
    response = bedrock_client.converse(
        modelId=model_id,
        messages=memory,
        toolConfig=tool_config
    )
    memory.append(response['output']['message'])

    stop_reason = response['stopReason']
    if stop_reason == "end_turn":
        break
    if stop_reason == "tool_use":
        tools_to_run = XpanderClient.extract_tool_calls(llm_response=response, llm_provider=LLMProvider.AMAZON_BEDROCK)
        for tool_to_run in tools_to_run:
            print("decided to run tool", tool_to_run.name)
        tool_responses = agent1.run_tools(tools_to_run)
        for tool_response in tool_responses:
            print("tool status code", tool_response.status_code)
            memory.append({"role": "user", "content": [{"toolResult": {"toolUseId": tool_response.tool_call_id, "content": [{"json": {"result": tool_response.result}}]}}]})

answer = response['output']['message']['content'][0]['text']
Markdown(answer)

decided to run tool LinkedInCompanyManagementGetCompanyInfoByDomain
tool status code 200
decided to run tool CrunchbaseOrganizationSearchAutocompleteByQuery
tool status code 400
decided to run tool tavily_insights_fetchInsightsFromTavilyAI
tool status code 200


Let me summarize the information about xpander.ai based on the research:

xpander.ai is a technology company that specializes in AI agent development and integration solutions. Here are the key points about the company and their technology:

Main Technology and What They Do:
1. AI Agent Building Platform: They develop a platform that enables the creation and deployment of intelligent AI agents capable of autonomous decision-making and task execution.

2. Key Technologies:
- AI-Ready Connector Technology: Their platform offers a connector generator that creates tailored integrations between AI and various systems
- Agent Graph System (AGS): A novel technology that improves the reliability and efficiency of AI agents
- Agentic Interfaces: Technology for handling complex multi-step AI agent workflows

Interesting Facts:
1. Their Agent Graph System (AGS) has been reported to improve GPT-4 success rates by 4x in handling complex tasks
2. The platform is designed for enterprise companies to deploy AI agents within their own systems
3. They work with major tech partners, including integration with NVIDIA NIM microservices
4. The company was founded by David Twizer (CEO) and Ran Sheinberg (Chief Product Officer)

Their Main Value Proposition:
- They help businesses automate complex tasks by enabling AI agents to seamlessly interact across different systems
- Their technology allows for autonomous execution of multi-step workflows
- They focus on making AI agent technology more accessible and reliable for enterprise use
- Their platform reduces the complexity of implementing AI solutions in real-world applications

The company appears to be at the forefront of AI agent technology, focusing on making AI implementations more practical and efficient for business use through their specialized connector and agent management systems.